In [1]:
from aipython.stripsProblem import STRIPS_domain, Planning_problem
from aipython.stripsForwardPlanner import Forward_STRIPS
from aipython.searchMPP import SearcherMPP
from lab02.aipython.stripsForwardPlanner import State


In [2]:
def format_state(state):
    """
    Formats a State object for more readable output.
    
    Args:
    state (State): The state to format.
    
    Returns:
    str: A formatted string representation of the state.
    """
    formatted_parts = []
    # Access the assignment attribute for state details
    assignment = state.assignment

    # Example formatting
    if 'scv-at' in assignment:
        formatted_parts.append(f"SCV is at {assignment['scv-at']}")
    if 'scv-has-minerals' in assignment and assignment['scv-has-minerals']:
        formatted_parts.append("SCV has minerals")
    else:
        formatted_parts.append("SCV does not have minerals")
    
    # Add more formatting based on the keys you care about
    for key, value in assignment.items():
        if key.startswith('building-at') and value != 'none':
            location = key.replace('building-at-', '')
            formatted_parts.append(f"{value} at {location}")
    
    return ", ".join(formatted_parts)

def print_solution_path(solution_path):
    if solution_path and isinstance(solution_path, Path):
        nodes = list(solution_path.nodes())  # Convert generator to list to reverse it
        nodes = list(reversed(nodes))  # Reverse to start from the initial state
        print("Solution Path:")
        for node in nodes:
            print(format_state(node))
        print(f"Total Cost: {solution_path.cost}")
    else:
        print("No solution found.")

In [3]:
from lab02.aipython.stripsProblem import Strips

class Strips_starcraft(object):
    def __init__(self, name, preconds, add_effects, del_effects=None, cost=1):
        """
        Defines the STRIPS representation for an action:
        - name is the name of the action
        - preconds, the preconditions, is a dictionary {feature: value} that must hold for the action to be carried out
        - add_effects is a dictionary {feature: value} of the positive effects of the action
        - del_effects is an optional dictionary {feature: value} of the negative effects of the action
        - cost is the cost of the action
        """
        self.name = name
        self.preconds = preconds
        self.add_effects = add_effects
        self.del_effects = del_effects if del_effects is not None else {}
        self.cost = cost

    def __repr__(self):
        return self.name

# Define the STRIPS domain
delivery_domain = STRIPS_domain(
    {
        'RLoc': {'cs', 'off', 'lab', 'mr'},  # Possible robot locations
        'RHC': {False, True},  # Robot has cargo?
        'SWC': {False, True},  # Warehouse has cargo?
        'MW': {False, True},   # Robot has maintenance need?
        'RHM': {False, True}   # Robot has maintenance mode?
    },
    {  # Set of STRIPS actions
        Strips('mc_cs', {'RLoc': 'cs'}, {'RLoc': 'off'}),
        Strips('mc_off', {'RLoc': 'off'}, {'RLoc': 'lab'}),
        Strips('mc_lab', {'RLoc': 'lab'}, {'RLoc': 'mr'}),
        Strips('mc_mr', {'RLoc': 'mr'}, {'RLoc': 'cs'}),
        Strips('mcc_cs', {'RLoc': 'cs'}, {'RLoc': 'mr'}),
        Strips('mcc_off', {'RLoc': 'off'}, {'RLoc': 'cs'}),
        Strips('mcc_lab', {'RLoc': 'lab'}, {'RLoc': 'off'}),
        Strips('mcc_mr', {'RLoc': 'mr'}, {'RLoc': 'lab'}),
        Strips('puc', {'RLoc': 'cs', 'RHC': False}, {'RHC': True}),
        Strips('dc', {'RLoc': 'off', 'RHC': True}, {'RHC': False, 'SWC': False}),
        Strips('pum', {'RLoc': 'mr', 'MW': True}, {'RHM': True, 'MW': False}),
        Strips('dm', {'RLoc': 'off', 'RHM': True}, {'RHM': False})
    }
)


def heuristic(state, goal):
    """
    Heuristic function to estimate the cost to reach the goal state from the given state.
    """
    # Calculate the number of unsatisfied goals
    unsatisfied_goals = 0
    for key, value in goal.items():
        if state[key] != value:
            unsatisfied_goals += 1
    return unsatisfied_goals

def format_path(path):
    """
    Format the path for readability.
    """
    formatted_steps = []
    for i, step in enumerate(path.steps):
        state = step.state
        action = step.action
        if i == 0:
            formatted_steps.append(f"Initial State: {state}")
        else:
            formatted_steps.append(f"Step {i}: Perform action '{action}'")
            formatted_steps.append(f"Resulting State: {state}")
    return "\n".join(formatted_steps)


## Problem 1

In [4]:
# Define the initial state
initial_state = {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}

# Define the goal state
goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj = State(initial_state)
goal_state_obj = State(goal_state)

# Calculate the heuristic value using the heuristic function and initial state
# heuristic_value = heuristic(initial_state_obj, goal_state_obj)

# Define the Planning problem
problem_1 = Planning_problem(delivery_domain, initial_state, goal_state)

# Instantiate the Forward_STRIPS class with the planning problem and heuristic function
forward_strips = Forward_STRIPS(problem_1)


# Use A* search algorithm to find a solution
solution = SearcherMPP(forward_strips).search()

# Print the solution
print(solution)

Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --puc--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --dc--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 7)
 25 paths have been expanded and 17 paths remain in the frontier
{'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SW

## Problem 2

In [5]:
initial_state_2 = { 'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False }

goal_state_2 = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_2 = State(initial_state)
goal_state_obj_2 = State(goal_state)

# Define the Planning problem
problem_2 = Planning_problem(delivery_domain, initial_state_2, goal_state_2)

# Solve the problem using the forward planner
forward_strips_2 = Forward_STRIPS(problem_2, heur=heuristic)
solution_2 = SearcherMPP(forward_strips_2).search()

    

Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mcc_mr--> {'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mcc_lab--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': False} (cost: 5)
 13 paths have been expanded and 8 paths remain in the frontier


## Problem 3

In [6]:
# Define the initial state and goal state
initial_state_3 = { 'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True }


goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_3 = State(initial_state)
goal_state_obj_3 = State(goal_state)

# Define the Planning problem
problem_3 = Planning_problem(delivery_domain, initial_state_3, goal_state)

# Solve the problem
forward_strips_3 = Forward_STRIPS(problem_3)
solution_3 = SearcherMPP(forward_strips_3).search()


Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mcc_mr--> {'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mcc_lab--> {'RLoc': 'off', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --dc--> {'RLoc': 'off', 'RHC': False, 'SWC': False, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': False, 'SWC': False, 'MW': False, 'RHM': False} (cost: 6)
 34 paths have been expanded and 20 paths remain in the frontier


# Harder problem

In [7]:
def gen_starcraft_move(from_loc, to_loc, unit):
    return 'move-' + from_loc + '-' + to_loc + '-' + unit


def gen_starcraft_collect_minerals(area, unit):
    return 'collect-minerals-' + area + '-' + unit


def gen_starcraft_build(unit, area, building):
    return 'build-' + unit + '-' + area + '-' + building


def gen_starcraft_train(builder, building, area):
    return 'train-unit-' + builder + '-' + building + '-' + area

# unit x at area
def at(x):
    return '' + x + '-at'

# minerals at area x depleted 
def min_depleted(x):
    return '' + x + '-empty'

# unit x has minerals
def has_minerals(x):
    return '' + x + '-has-minerals'

# building at area x
def building_at(x):
    return 'building-at-' + x


def build_starcraft_domain():
    areas = {'sectorA', 'sectorB', 'sectorC', 'mineralFieldA', 'mineralFieldB', 'mineralFieldC', 'mineralFieldD'}
    buildings = {'barracks', 'factory', 'starport', 'supplyDepot', 'fusionCore'}
    units = {'scv', 'marine', 'tank', 'battlecruiser'}

    actions = set()

    # MOVE action
    for from_area in areas:
        for to_area in areas - {from_area}:
            for unit in units:
                name = gen_starcraft_move(from_area, to_area, unit)
                preconds = {at(unit): from_area}
                effects = {at(unit): to_area}
                actions.add(Strips(name, preconds, effects))

    # COLLECT MINERALS action
    # Restrict to mineral fields
    for area in areas - {'sectorA', 'sectorB', 'sectorC'}: 
        name = gen_starcraft_collect_minerals(area, 'scv')
        preconds = {at('scv'): area, min_depleted(area): False}
        effects = {has_minerals('scv'): True, min_depleted(area): True}
        actions.add(Strips(name, preconds, effects))

    # BUILD action for each building type
    for area in areas - {'mineralFieldA', 'mineralFieldB', 'mineralFieldC', 'mineralFieldD'}:
        for building in buildings:
            name = gen_starcraft_build('scv', area, building)
            preconds = {at('scv'): area, has_minerals('scv'): True, building_at(area): 'none'}
            effects = {building_at(area): building, has_minerals('scv'): False, f'has-{building}': True}
            if building != 'supplyDepot':  # Assuming supply depot is the base requireme nt for other buildings
                preconds['has-supplyDepot'] = True
            actions.add(Strips(name, preconds, effects))

    # TRAIN action for units
    # Assuming specific buildings are required for training specific units
    train_requirements = {'marine': 'barracks', 'tank': 'factory', 'wraith': 'starport', 'battleCruiser': 'fusionCore'}
    for unit, required_building in train_requirements.items():
        for area in areas:
            name = gen_starcraft_train('scv', required_building, area)
            preconds = {has_minerals('scv'): True, building_at(area): required_building}
            effects = {at(unit): area, has_minerals('scv'): False}  # Assuming training consumes minerals
            actions.add(Strips(name, preconds, effects))

    feature_domain_dict = {
        'at-scv': areas,
        'scv-has-minerals': {True, False},
        'minerals-depleted': {True, False},
        'building-at': {**{area: 'none' for area in areas}, **{area: buildings for area in areas}},
    }

    return STRIPS_domain(feature_domain_dict, actions)

starcraft_domain = build_starcraft_domain()


In [8]:
from lab02.aipython.searchProblem import Path

# Starcraft Problem 1
# Train marine

initial_state_marine = {
    at('scv'): 'sectorA',  # SCV starts in sectorA
    at('marine'): 'none',  # Marine is not trained initially
    at('battlecruiser'): 'none',  # Battlecruiser is not trained initially
    at('tank'): 'none',  # Tank is not trained initially
    min_depleted('sectorA'): True,
    min_depleted('sectorB'): True,
    min_depleted('sectorC'): True,
    min_depleted('mineralFieldA'): False,  # Mineral fields are not depleted
    min_depleted('mineralFieldB'): False,
    min_depleted('mineralFieldC'): False,
    min_depleted('mineralFieldD'): False,
    has_minerals('scv'): False,  # SCV starts without minerals
    building_at('sectorA'): 'none',  # No buildings in sectorA initially
    building_at('sectorB'): 'none',  # No buildings in sectorB initially
    building_at('sectorC'): 'none',  # No buildings in sectorC initially
    building_at('mineralFieldA'): 'none',  # No buildings in 
    building_at('mineralFieldB'): 'none',
    building_at('mineralFieldC'): 'none',
    building_at('mineralFieldD'): 'none',
    'has-supplyDepot': False,
    'has-barracks': False,
}

goal_state_marine = {
    at('marine'): 'sectorA',  # Marine should be trained in sectorA
}

problem_4 = Planning_problem(starcraft_domain, initial_state_marine, goal_state_marine)

forward_strips_4 = Forward_STRIPS(problem_4)

solution_4 = SearcherMPP(forward_strips_4).search()

print_solution_path(solution_4)


Solution: {'scv-at': 'sectorA', 'marine-at': 'none', 'battlecruiser-at': 'none', 'tank-at': 'none', 'sectorA-empty': True, 'sectorB-empty': True, 'sectorC-empty': True, 'mineralFieldA-empty': False, 'mineralFieldB-empty': False, 'mineralFieldC-empty': False, 'mineralFieldD-empty': False, 'scv-has-minerals': False, 'building-at-sectorA': 'none', 'building-at-sectorB': 'none', 'building-at-sectorC': 'none', 'building-at-mineralFieldA': 'none', 'building-at-mineralFieldB': 'none', 'building-at-mineralFieldC': 'none', 'building-at-mineralFieldD': 'none', 'has-supplyDepot': False, 'has-barracks': False}
   --move-sectorA-mineralFieldC-scv--> {'scv-at': 'mineralFieldC', 'marine-at': 'none', 'battlecruiser-at': 'none', 'tank-at': 'none', 'sectorA-empty': True, 'sectorB-empty': True, 'sectorC-empty': True, 'mineralFieldA-empty': False, 'mineralFieldB-empty': False, 'mineralFieldC-empty': False, 'mineralFieldD-empty': False, 'scv-has-minerals': False, 'building-at-sectorA': 'none', 'building-at

In [16]:
# Starcraft Problem 2
# build tank


goal_state_tank = {
    building_at('sectorA'): 'factory',  # Factory should be built in sectorA
    building_at('sectorB'): 'barracks',  # Barracks should be built in sectorA
    building_at('sectorC'): 'supplyDepot',  # Supply depot should be built in sectorB
    at('tank'): 'sectorC',
}

goal_state_5 = State(goal_state_tank)

problem_5 = Planning_problem(starcraft_domain, initial_state_marine, goal_state_tank)

forward_strips_5 = Forward_STRIPS(problem_5)

solution_5 = SearcherMPP(forward_strips_5).search()

print_solution_path(solution_5)


Solution: {'scv-at': 'sectorA', 'marine-at': 'none', 'battlecruiser-at': 'none', 'tank-at': 'none', 'sectorA-empty': True, 'sectorB-empty': True, 'sectorC-empty': True, 'mineralFieldA-empty': False, 'mineralFieldB-empty': False, 'mineralFieldC-empty': False, 'mineralFieldD-empty': False, 'scv-has-minerals': False, 'building-at-sectorA': 'none', 'building-at-sectorB': 'none', 'building-at-sectorC': 'none', 'building-at-mineralFieldA': 'none', 'building-at-mineralFieldB': 'none', 'building-at-mineralFieldC': 'none', 'building-at-mineralFieldD': 'none', 'has-supplyDepot': False, 'has-barracks': False}
   --move-sectorA-mineralFieldB-scv--> {'scv-at': 'mineralFieldB', 'marine-at': 'none', 'battlecruiser-at': 'none', 'tank-at': 'none', 'sectorA-empty': True, 'sectorB-empty': True, 'sectorC-empty': True, 'mineralFieldA-empty': False, 'mineralFieldB-empty': False, 'mineralFieldC-empty': False, 'mineralFieldD-empty': False, 'scv-has-minerals': False, 'building-at-sectorA': 'none', 'building-at

In [ ]:
# Starcraft Problem 3
# build battlecruiser

goal_state_three = {
    building_at('sectorA'): 'factory',  # Factory should be built in sectorA
    building_at('sectorB'): 'barracks',  # Barracks should be built in sectorA
    building_at('sectorC'): 'supplyDepot',  # Supply depot should be built in sectorB
    at('tank'): 'sectorC',
}

def advanced_heuristic(state, goal):
    cost = 0

    for unit in ['marine', 'tank', 'battlecruiser']:
        if f'at-{unit}' in goal and goal[f'at-{unit}'] != 'none' and state[f'at-{unit}'] == 'none':
            base_cost = {'marine': 1, 'tank': 2, 'battlecruiser': 3}
            cost += base_cost[unit]
            if not state['has_minerals(scv)']:
                cost += 1
            if unit == 'tank' and not state['has-factory']:
                cost += 1  
            if unit == 'battlecruiser' and (not state['has-starport'] or not state['has-fusionCore']):
                cost += 2  

    for building in ['supplyDepot', 'barracks', 'factory', 'starport', 'fusionCore']:
        if f'has-{building}' in goal and goal[f'has-{building}'] and not state[f'has-{building}']:
            cost += 1  

    if 'at-scv' in goal and goal['at-scv'] != state['at-scv']:
        cost += 1  

    return cost

goal_state_6 = State(goal_state_three)

problem_6 = Planning_problem(starcraft_domain, initial_state_marine, goal_state_three)

forward_strips_6 = Forward_STRIPS(problem_6, heur=advanced_heuristic)

solution_6 = SearcherMPP(forward_strips_6).search()

print_solution_path(solution_6)